In [1]:
!pip install gymnasium
!pip install "gymnasium[atari, accept-rom-license]"
!apt-get install -y swig
!pip install gymnasium[box2d]

In [2]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.autograd as autograd
from torch.autograd import Variable
from collections import deque, namedtuple

In [3]:
# Building the AI

# Background
# Possible Actions
# There are four discrete actions available:

# 0: do nothing
# 1: fire left orientation engine (move left)
# 2: fire main engine (move in the same direction)
# 3: fire right orientation engine (move right)

# State of the agent
# The state is an 8-dimensional vector:
# the coordinates of the lander in x & y
# linear velocities in x & y,
# angle, its angular velocity
# two booleans that represent whether each leg is in contact with the ground or not

# Rewards


In [4]:
# Creating the architecture of the network
class Network(nn.Module):
  # number of input neurons = size of the dimensions of the state (8)
  # number of actions
  # random seed
  def __init__(self, state_size, action_size, seed = 42):
    super(Network, self).__init__()
    # Sets the seed for generating random numbers
    self.seed = torch.manual_seed(seed)
    # first full connection between the input layer and hidden layer
    self.fc1 = nn.Linear(state_size, 64)
    # second full connection layer between first hidden layer and second layer
    self.fc2 = nn.Linear(64, 64)
    # connetion between the second hidden layer and the output layer
    self.fc3 = nn.Linear(64, action_size)

  # forward propogation from input layer to the output layer
  def forward(self, state):
    # applying activation function (propogate the signal from input layer
    # to the first hidden layer applying activation function)
    x = self.fc1(state)
    x = F.relu(x)
    x = self.fc2(x)
    x = F.relu(x)
    return self.fc3(x)

In [5]:
# Training the AI

In [6]:
# Setting up the environment
import gymnasium as gym
env = gym.make('LunarLander-v2')
# shape of the state object
state_shape = env.observation_space.shape
# size of the state (8 elements)
state_size = env.observation_space.shape[0]
number_actions = env.action_space.n
print('State shape: ',state_shape)
print('State size: ',state_size)
print('Number of actions: ',number_actions)

State shape:  (8,)
State size:  8
Number of actions:  4


In [7]:
# Initializing the hyper parameters
learning_rate = 5e-4
# size of each batch where the model will be trained
minibatch_size = 100
discount_factor = 0.99
# size of the replay buffer
replay_buffer_size = int(1e5)
interpolation_parameter = 1e-3

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [8]:
# Implementing experience replay
class ReplayMemory(object):
  # capacity -> size of the buffer
  def __init__(self, capacity):
    self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    self.capacity = capacity
    self.memory = []

  # append a new transition in the memory and ensures that memory contain
  # only 100000 transitions
  def push(self, event):
    self.memory.append(event)
    # check if buffer does not exceed the capacity
    if len(self.memory) > self.capacity:
      del self.memory[0]

  # batch_size -> number of experiences sampled in a batch
  # each experience contains (state,action,reward,nextstate,boolean done)
  def sample(self, batch_size):
    experiences = random.sample(self.memory, k=batch_size)
    # all the states corresponding to all experiences
    # next converting to torch tensor
    # finally move to the designated computing device
    states = torch.from_numpy(np.vstack([e[0] for e in experiences if e is not None])).float().to(self.device)
    # actions
    actions = torch.from_numpy(np.vstack([e[1] for e in experiences if e is not None])).long().to(self.device)
    # rewards
    rewards = torch.from_numpy(np.vstack([e[2] for e in experiences if e is not None])).float().to(self.device)
    # next states
    next_states = torch.from_numpy(np.vstack([e[3] for e in experiences if e is not None])).float().to(self.device)
    # dones
    dones = torch.from_numpy(np.vstack([e[4] for e in experiences if e is not None]).astype(np.uint8)).float().to(self.device)
    return states, next_states, actions, rewards, dones




In [9]:
# Implementing a DQN class
class Agent():

  def __init__(self, state_size, action_size):
    self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    self.state_size = state_size
    self.action_size = action_size

    # local q network
    self.local_qnetwork = Network(state_size, action_size).to(self.device)
    self.target_qnetwork = Network(state_size, action_size).to(self.device)
    # parameters are the weights of the network
    self.optimizer = optim.Adam(self.local_qnetwork.parameters(), lr=learning_rate)
    # replay memory
    self.memory = ReplayMemory(replay_buffer_size)
    # timestep to decide when to learn from the experirences
    self.t_step = 0

  # method to store exp and decide when to learn from them
  def step(self, state, action, reward, next_state, done):
    self.memory.push((state, action, reward, next_state, done))
    # when timestep reaches 4 the model will learn by taking a minibatch from the
    # replay buffer
    self.t_step = (self.t_step + 1) % 4
    if self.t_step == 0:
      # check if there are at least 100 exp in the buffer
      if len(self.memory.memory) > minibatch_size:
        experiences = self.memory.sample(100)
        self.learn(experiences, discount_factor)

  # select an action based on given state based on epsilon greedy method (not softmax)
  def act(self, state, epsilon = 0.):
    # adding an extra dimension corresponding to the batch (it indicates to which batch this state belogns to)
    # note that always the batch index should be at the beginning
    state = torch.from_numpy(state).float().unsqueeze(0).to(self.device)
    # set the local network to evaluation mode before forward pass
    # because as this is the forward pass we are making predictions
    self.local_qnetwork.eval()
    # since we are in forward there is no need to calculate gradients
    with torch.no_grad():
      # predicting q values (forward pass)
      action_values = self.local_qnetwork(state)
    # resetting model to traning mode
    self.local_qnetwork.train()
    # select an action based on epsilon greedy policy
    # we generate a random number R and if R > epsilon ? we choose the maximum predicted q value
    # : select a random aciton
    if random.random() > epsilon:
      # argmax returns the index of maxium value of a numpy array
      # .data selects the tensor data and .numpy() is used to convert tensor to numpy array
      return np.argmax(action_values.cpu().data.numpy())
    else:
      # selects a random index 0,1,2,3
      return random.choice(np.arange(self.action_size))

  # allows agent to learn based on the minibatch
  def learn(self, experiences, discount_factor):
    states, next_states, actions, rewards, dones = experiences
    # to compute the target q value we need the maxium q value for the next state
    # use the target network to get the q values for all the actions from that next state
    next_q_targets = self.target_qnetwork(next_states).detach().max(1)[0].unsqueeze(1)
    q_targets = rewards + (discount_factor * next_q_targets * (1 - dones))
    # forward propogate the states to get the predicted q values
    q_expected = self.local_qnetwork(states).gather(1, actions)
    # loss (mean squared error)
    loss = F.mse_loss(q_expected, q_targets)
    # backpropogating the error to update the weights
    self.optimizer.zero_grad()
    loss.backward()
    # single optimization step for updating the weights
    self.optimizer.step()
    # updating the target network weights
    self.soft_update(self.local_qnetwork, self.target_qnetwork, interpolation_parameter)

  def soft_update(self, local_model, target_model, interpolation_parameter):
    for target_param, local_param in zip(target_model.parameters(),local_model.parameters()):
      # softly update the target model parameters with the weighted average of the local and target params
      target_param.data.copy_(interpolation_parameter * local_param.data + (1.0 - interpolation_parameter) * target_param.data)

  def load(self):
    if os.path.isfile('checkpoint.pth'):
      print("=> loading checkpoint...")
      checkpoint = torch.load('checkpoint.pth')
      self.local_qnetwork.load_state_dict(checkpoint)
      print('Done!')

In [10]:
# Initiazling the DQN agent
agent = Agent(state_size, number_actions)
agent.load()

=> loading checkpoint...
Done!


In [11]:
# Training the DQN Agent

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [12]:
number_episodes = 2000
maximum_number_timesteps_per_episode = 1000
epsilon_starting_value = 1.0
epsilon_ending_value = 0.01
epsilon_decay_value = 0.995
epsilon = epsilon_starting_value
scores_on_100_episodes = deque(maxlen=100)

for episode in range(1, number_episodes + 1):
  state, _ = env.reset()
  score = 0
  for t in range(0, maximum_number_timesteps_per_episode):
    action = agent.act(state, epsilon)
    next_state, reward, done, _, _ = env.step(action)
    agent.step(state, action, reward, next_state, done)
    state = next_state
    score += reward
    if done:
      break
  scores_on_100_episodes.append(score)
  epsilon = max(epsilon_ending_value, epsilon_decay_value * epsilon)
  print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode,np.mean(scores_on_100_episodes)),end="")
  if episode % 100 == 0:
    print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode,np.mean(scores_on_100_episodes)))
  if np.mean(scores_on_100_episodes) >= 200.0:
    print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(episode - 100, np.mean(scores_on_100_episodes)))
    torch.save(agent.local_qnetwork.state_dict(), 'checkpoint.pth')
    break


In [13]:
import os

In [14]:
import glob
import io
import base64
import imageio
from IPython.display import HTML, display
from gym.wrappers.monitoring.video_recorder import VideoRecorder

def show_video_of_model(agent, env_name):
    env = gym.make(env_name, render_mode='rgb_array')
    state, _ = env.reset()
    done = False
    frames = []
    while not done:
        frame = env.render()
        frames.append(frame)
        action = agent.act(state)
        state, reward, done, _, _ = env.step(action.item())
    env.close()
    imageio.mimsave('video.mp4', frames, fps=30)

show_video_of_model(agent, 'LunarLander-v2')

def show_video():
    mp4list = glob.glob('*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        display(HTML(data='''<video alt="test" autoplay
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
    else:
        print("Could not find video")

show_video()